# **Bayesian Optimization을 활용한 딥러닝 하이퍼파라미터 최적화**

## SECOM 반도체 불량 예측 모델

### 학습 목표
1. Bayesian Optimization의 원리 이해 (Surrogate Model, Acquisition Function)
2. 딥러닝 모델의 하이퍼파라미터 최적화 실습
3. Grid Search vs Bayesian Optimization 성능 비교
4. 불균형 데이터셋에서의 모델 성능 향상

### 데이터셋: SECOM (SEmiCOnductor Manufacturing)
- **샘플 수**: 1,567개
- **특성 수**: 590개 (센서 데이터)
- **타겟**: Pass(-1) / Fail(1) - 약 6.6% 불량률
- **특징**: 고차원, 결측치 존재, 심한 클래스 불균형

## 1. 환경 설정 및 라이브러리 임포트

In [ ]:
# conda 환경 생성
# conda create -n pj_bo python=3.10
# conda activate pj_bo
# ls # 현재 directory 위치 확인을 위한 bash 명령어
# cd Project_1_BO # 프로젝트 폴더 내로 이동하는 명령어

In [ ]:
# 필요한 라이브러리 설치
# pip install optuna torch scikit-learn pandas matplotlib seaborn plotly jupyter notebook
# python -m ipykernel install --user --name pj_bo --display-name pj_bo

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    f1_score, precision_score, recall_score, 
    confusion_matrix, classification_report, roc_auc_score
)

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Bayesian Optimization
import optuna
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate
)

# 재현성을 위한 시드 설정
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. 데이터 로딩 및 전처리

In [ ]:
# SECOM 데이터 로딩 (uci-secom.csv)
df = pd.read_csv('./uci-secom.csv')

# Time 컬럼과 Pass/Fail 컬럼 분리
X = df.drop(columns=['Time', 'Pass/Fail'])
y = df['Pass/Fail'].values

# 레이블 변환: -1 -> 0 (Pass), 1 -> 1 (Fail)
y = np.where(y == -1, 0, 1)

print("=" * 50)
print("SECOM 데이터셋 정보")
print("=" * 50)
print(f"데이터 형태: {X.shape}")
print(f"Pass (0): {(y == 0).sum()} ({(y == 0).sum() / len(y) * 100:.2f}%)")
print(f"Fail (1): {(y == 1).sum()} ({(y == 1).sum() / len(y) * 100:.2f}%)")
print(f"결측치 비율: {X.isnull().sum().sum() / X.size * 100:.2f}%")

### 2.1 데이터프레임 기본 정보 확인

In [ ]:
# 데이터프레임 상위 5개 행 확인
print("데이터프레임 상위 5개 행:")


In [ ]:
# 데이터프레임 기본 정보
print("데이터프레임 정보:")

print(f"\n컬럼 타입 분포:")

print(f"\n메모리 사용량: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# 기술 통계량 확인
print("기술 통계량 (일부 컬럼):")
df.iloc[:, 1:11].describe()

### 2.2 타겟 변수 (Pass/Fail) 분포 분석

In [ ]:
# 타겟 변수 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 클래스 분포 (원본 레이블)


# 파이 차트


plt.tight_layout()
plt.show()

print(f"\n클래스 불균형 비율: 1:{class_counts[-1]/class_counts[1]:.1f} (Pass:Fail)")

### 2.3 결측치 분석

In [ ]:
# 결측치 분석
missing_per_col = X.isnull().sum()
missing_ratio_per_col = X.isnull().sum() / len(X) * 100

# 결측치가 있는 컬럼만 필터링
cols_with_missing = missing_per_col[missing_per_col > 0]


# 결측치 비율 분포
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 결측치 비율 히스토그램


# 상위 20개 결측치 컬럼 시각화
top_missing = missing_ratio_per_col.nlargest(20)


plt.tight_layout()
plt.show()

### 2.4 특성 분포 시각화

In [ ]:
# 일부 특성의 분포 시각화 (결측치가 적은 컬럼 선택)
low_missing_cols = missing_ratio_per_col[missing_ratio_per_col < 5].index[:12].tolist()

fig, axes = plt.subplots(3, 4, figsize=(16, 10))
axes = axes.flatten()


plt.suptitle('Feature Distributions by Class (Pass vs Fail)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

### 2.5 특성 간 상관관계 분석

In [ ]:
# 상관관계 분석 (일부 특성) - Correlation Matrix 를 통해 시각화
# 결측치가 적은 상위 30개 컬럼 선택
sample_cols = missing_ratio_per_col[missing_ratio_per_col < 5].index[:30].tolist()
corr_matrix = X[sample_cols].corr()


plt.tight_layout()
plt.show()

# 높은 상관관계를 가진 특성 쌍 찾기


print(f"\n높은 상관관계 (|r| > 0.9)를 가진 특성 쌍: {len(high_corr_pairs)}개")
if high_corr_pairs[:5]:
    print("예시:")
    for col1, col2, corr in high_corr_pairs[:5]:
        print(f"  Feature {col1} - Feature {col2}: {corr:.3f}")

### 2.6 EDA 요약

In [ ]:
# EDA 요약
print("=" * 60)
print("EDA 요약")
print("=" * 60)
print(f"\n[데이터 크기]")
print(f"  - 샘플 수: { }")
print(f"  - 특성 수: { }")

print(f"\n[클래스 분포]")
print(f"  - Pass: { }%)")
print(f"  - Fail: { }%)")
print(f"  - 불균형 비율: 1:{ }")

print(f"\n[결측치]")
print(f"  - 결측치 있는 컬럼: { }개")
print(f"  - 전체 결측치 비율: { }%")
print(f"  - 50% 이상 결측 컬럼: { }개")

print(f"\n[주요 특징]")
print(f"  **직접 작성해보세요**")

print("=" * 60)

### 2.7 데이터 전처리

In [ ]:
# 결측치 비율이 50% 이상인 컬럼 제거


# 결측치 대치 (평균값)

# 표준화 - Standard Scaler 활용


print(f"전처리 완료 데이터 형태: {X_scaled.shape}")

In [ ]:
# 학습/검증/테스트 데이터 분할

# temp(train,val), test를 8:2로 분할
X_temp, X_test, y_temp, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=SEED, stratify=y
)

# temp 내에서 train과 val을 8:2로 분할
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=SEED, stratify=y_temp
)

print(f"Train: {X_train.shape[0]} samples (Fail: {y_train.sum()})")
print(f"Val:   {X_val.shape[0]} samples (Fail: {y_val.sum()})")
print(f"Test:  {X_test.shape[0]} samples (Fail: {y_test.sum()})")

In [ ]:
# PyTorch Tensor 변환
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
X_val_tensor = torch.FloatTensor(X_val).to(device)
y_val_tensor = torch.FloatTensor(y_val).to(device)
X_test_tensor = torch.FloatTensor(X_test).to(device)
y_test_tensor = torch.FloatTensor(y_test).to(device)

# 클래스 가중치 계산 (불균형 처리)
n_negative = (y_train == 0).sum()
n_positive = (y_train == 1).sum()
pos_weight = torch.tensor([n_negative / n_positive]).to(device)
print(f"Positive class weight: {pos_weight.item():.2f}")

## 3. 딥러닝 모델 정의

다양한 하이퍼파라미터를 조절할 수 있는 유연한 MLP 모델을 정의합니다.

In [ ]:
class FlexibleMLP(nn.Module):
    """
    유연한 MLP 모델 - Bayesian Optimization을 위한 하이퍼파라미터 조절 가능
    
    Parameters:
    - input_dim: 입력 차원
    - hidden_dims: 은닉층 차원 리스트 (예: [256, 128, 64])
    - dropout_rate: 드롭아웃 비율
    - use_batch_norm: 배치 정규화 사용 여부
    """
    def __init__(self, input_dim, hidden_dims, dropout_rate=0.3, use_batch_norm=True):
        super(FlexibleMLP, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        # MLP 내부 은닉층의 구축

        
        # 출력층

        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, 
                lr, batch_size, epochs, pos_weight, early_stopping_patience=10):
    """
    모델 학습 함수 (Early Stopping 포함)
    """
    # DataLoader 생성
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # 손실 함수 및 옵티마이저
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    best_val_f1 = 0
    patience_counter = 0
    
    for epoch in range(epochs):
        # Training
        
        
        
        # Validation
        
        
        # Early Stopping
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                break
    
    return best_val_f1

## 4. Bayesian Optimization 이론

### Bayesian Optimization이란?
- **목적**: 평가 비용이 높은 블랙박스 함수의 전역 최적값을 효율적으로 찾는 방법
- **핵심 아이디어**: 이전 평가 결과를 바탕으로 다음에 평가할 지점을 "지능적으로" 선택

### 주요 구성 요소

#### 1. Surrogate Model (대리 모델)
- 실제 목적 함수를 근사하는 확률적 모델
- 주로 **Gaussian Process (GP)** 또는 **Tree-structured Parzen Estimator (TPE)** 사용
- 평균과 불확실성(분산)을 동시에 제공

#### 2. Acquisition Function (획득 함수)
- 다음에 탐색할 지점을 결정하는 함수
- **Exploitation**: 현재까지 좋은 성능을 보인 영역 탐색
- **Exploration**: 불확실성이 높은 영역 탐색
- 대표적인 획득 함수:
  - Expected Improvement (EI)
  - Upper Confidence Bound (UCB)
  - Probability of Improvement (PI)

### Optuna의 TPE (Tree-structured Parzen Estimator)
- 조건부 확률을 활용한 베이지안 최적화 방법
- `p(x|y)` 대신 `p(y|x)`를 모델링
- 고차원 공간에서 효율적

## 5. Optuna를 이용한 Bayesian Optimization 구현

In [ ]:
def objective(trial):
    """
    Optuna Objective Function
    
    최적화할 하이퍼파라미터:
    1. 네트워크 구조: 레이어 수, 각 레이어의 뉴런 수
    2. 정규화: Dropout rate
    3. 학습: Learning rate, Batch size
    """
    
    # 1. 네트워크 구조 하이퍼파라미터
    n_layers = trial.suggest_int('n_layers', 2, 5)
    hidden_dims = []
    for i in range(n_layers):
        hidden_dim = trial.suggest_int(f'hidden_dim_{i}', 32, 512, step=32)
        hidden_dims.append(hidden_dim)
    
    # 2. 정규화 하이퍼파라미터
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    use_batch_norm =  True 

    # 3. 학습 하이퍼파라미터
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    
    # 모델 생성
    model = FlexibleMLP(
        input_dim=X_train.shape[1],
        hidden_dims=hidden_dims,
        dropout_rate=dropout_rate,
        use_batch_norm=use_batch_norm
    ).to(device)
    
    # 모델 학습 및 검증 F1 스코어 반환
    val_f1 = train_model(
        model, X_train_tensor, y_train_tensor,
        X_val_tensor, y_val_tensor,
        lr=lr, batch_size=batch_size, epochs=100,
        pos_weight=pos_weight, early_stopping_patience=15
    )
    
    return val_f1

In [ ]:
# Optuna Study 생성 및 최적화 실행
print("=" * 50)
print("Bayesian Optimization 시작 (Optuna TPE)")
print("=" * 50)

# TPE Sampler 사용 (Bayesian Optimization)
sampler = optuna.samplers.TPESampler(seed=SEED)

study = optuna.create_study(
    direction='maximize',  # F1 스코어 최대화
    sampler=sampler,
    study_name='secom_dnn_optimization'
)

# 최적화 실행 (50회 시도)
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\n" + "=" * 50)
print("최적화 완료!")
print("=" * 50)
print(f"Best F1 Score: {study.best_value:.4f}")
print(f"\nBest Hyperparameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

## 6. 최적화 결과 시각화

#### Optuna의 시각화 함수 사용
- plot_optimization_history: 최적화 진행 과정을 시각화하여 각 시도마다 목적 함수(F1-score)이 어떻게 변화하고 개선되었는지 시각화
- plot_param_importances: 하이퍼파라미터의 중요도를 바 차트로 시각화. 전체 성능에 가장 큰 영향을 미친 파라미터에 대한 분석
- plot_parallel_coordinate: 하이퍼파라미터 간의 관계와 목적 함수 값의 분포를 평행 좌표계 그래프로 시각화. 좋은 성능을 낸 파라미터 값들의 패턴 파악 가능

In [ ]:
# 1. 최적화 히스토리
fig = 

In [ ]:
# 2. 하이퍼파라미터 중요도
fig = 

In [ ]:
# 3. Parallel Coordinate Plot
fig = 

In [ ]:
# 4. 최적화 과정 시각화 (matplotlib)
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Trial별 F1 Score
trials_df = study.trials_dataframe()
ax1 = axes[0, 0]
ax1.plot(trials_df['number'], trials_df['value'], 'b-o', alpha=0.6, markersize=4)
ax1.axhline(y=study.best_value, color='r', linestyle='--', label=f'Best: {study.best_value:.4f}')
ax1.set_xlabel('Trial')
ax1.set_ylabel('F1 Score')
ax1.set_title('Optimization Progress')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Learning Rate 분포
ax2 = axes[0, 1]
ax2.scatter(trials_df['params_lr'], trials_df['value'], c=trials_df['number'], cmap='viridis', alpha=0.6)
ax2.axvline(x=study.best_params['lr'], color='r', linestyle='--', label=f'Best LR: {study.best_params["lr"]:.6f}')
ax2.set_xscale('log')
ax2.set_xlabel('Learning Rate')
ax2.set_ylabel('F1 Score')
ax2.set_title('Learning Rate vs F1 Score')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Dropout Rate 분포
ax3 = axes[1, 0]
ax3.scatter(trials_df['params_dropout_rate'], trials_df['value'], c=trials_df['number'], cmap='viridis', alpha=0.6)
ax3.axvline(x=study.best_params['dropout_rate'], color='r', linestyle='--', label=f'Best Dropout: {study.best_params["dropout_rate"]:.2f}')
ax3.set_xlabel('Dropout Rate')
ax3.set_ylabel('F1 Score')
ax3.set_title('Dropout Rate vs F1 Score')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Number of Layers 분포
ax4 = axes[1, 1]
layer_performance = trials_df.groupby('params_n_layers')['value'].agg(['mean', 'std', 'max'])
ax4.bar(layer_performance.index, layer_performance['mean'], yerr=layer_performance['std'], 
        alpha=0.7, capsize=5, color='steelblue')
ax4.scatter(layer_performance.index, layer_performance['max'], color='red', s=100, zorder=5, label='Best')
ax4.set_xlabel('Number of Layers')
ax4.set_ylabel('F1 Score')
ax4.set_title('Performance by Number of Layers')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. 최적 모델로 최종 학습 및 테스트 평가

In [ ]:
# 최적 하이퍼파라미터로 모델 재생성
best_params = study.best_params

# Hidden dimensions 추출


print("=" * 50)
print("최적 모델 구성")
print("=" * 50)
print(f"Hidden Layers: {hidden_dims}")
print(f"Dropout Rate: {best_params['dropout_rate']:.3f}")
print(f"Learning Rate: {best_params['lr']:.6f}")
print(f"Batch Size: {best_params['batch_size']}")

In [ ]:
# Train + Validation 데이터로 최종 학습
X_train_full = np.vstack([X_train, X_val])
y_train_full = np.concatenate([y_train, y_val])

X_train_full_tensor = torch.FloatTensor(X_train_full).to(device)
y_train_full_tensor = torch.FloatTensor(y_train_full).to(device)

# 최적 모델 생성
best_model = FlexibleMLP(
    # 최적화된 하이퍼파라미터로 모델 구축
).to(device)

print(best_model)

In [ ]:
# 최종 학습
train_dataset = TensorDataset(X_train_full_tensor, y_train_full_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(best_model.parameters(), lr=best_params['lr'])

# 학습 히스토리 저장
history = {'train_loss': [], 'train_f1': []}

epochs = 150
for epoch in range(epochs):
    best_model.train()
    epoch_loss = 0
    
    # 학습 코드
    
    # 에폭별 메트릭 계산
    best_model.eval()
    with torch.no_grad():
        train_outputs = best_model(X_train_full_tensor).squeeze()
        train_preds = (torch.sigmoid(train_outputs) > 0.5).float().cpu().numpy()
        train_f1 = f1_score(y_train_full, train_preds, zero_division=0)
    
    history['train_loss'].append(epoch_loss / len(train_loader))
    history['train_f1'].append(train_f1)
    
    if (epoch + 1) % 30 == 0:
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {history['train_loss'][-1]:.4f}, F1: {train_f1:.4f}")

In [ ]:
# 학습 곡선 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss 학습 곡선 시각화


# F1 score 학습 곡선 시각화


plt.tight_layout()
plt.show()

In [ ]:
# 테스트 세트 평가
best_model.eval()
with torch.no_grad():

print("=" * 50)
print("최종 테스트 결과 (Bayesian Optimization 적용 모델)")
print("=" * 50)
print(f"\nClassification Report:")
print(classification_report(y_test, test_preds, target_names=['Pass', 'Fail']))

print(f"\nMetrics Summary:")
print(f"  F1 Score:    {}")
print(f"  Precision:   {}")
print(f"  Recall:      {}")
print(f"  ROC-AUC:     {}")

In [ ]:
# Confusion Matrix 시각화 - acutal pass와 predicive pass 간의 matrix

# Confusion Matrix 해석표 작성
print(f"\nConfusion Matrix 해석:")
print(f"  True Negative (Pass 정확): { }")
print(f"  False Positive (Pass->Fail 오분류): { }")
print(f"  False Negative (Fail->Pass 오분류): { }")
print(f"  True Positive (Fail 정확): { }")

## 8. 기본 모델 vs 최적화 모델 비교

In [ ]:
# 기본 하이퍼파라미터 모델 (최적화 전)
baseline_model = FlexibleMLP(
    input_dim=X_train.shape[1],
    hidden_dims=[256, 128, 64],  # 임의의 기본값
    dropout_rate=0.3,
    use_batch_norm=True
).to(device)

# 기본 모델 학습
baseline_dataset = TensorDataset(X_train_full_tensor, y_train_full_tensor)
baseline_loader = DataLoader(baseline_dataset, batch_size=64, shuffle=True)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(baseline_model.parameters(), lr=0.001)

# 기본 모델 학습 코드

# 기본 모델 평가
baseline_model.eval()


print("=" * 50)
print("Baseline 모델 테스트 결과 (최적화 전)")
print("=" * 50)
print(classification_report(y_test, baseline_preds, target_names=['Pass', 'Fail']))

In [ ]:
# 성능 비교 시각화
metrics = ['F1 Score', 'Precision', 'Recall', 'ROC-AUC']

baseline_scores = [
    # f1_score
    # precision score
    # recall score
    # roc_auc score
]

optimized_scores = [
    # f1_score
    # precision score
    # recall score
    # roc_auc score
]

x = np.arange(len(metrics))

# bar 그래프 시각화
ax.set_ylabel('Score')
ax.set_title('Baseline vs Bayesian Optimization Model Performance')


# 값 표시
for bar, score in zip(bars1, baseline_scores):
    

for bar, score in zip(bars2, optimized_scores):
    

plt.tight_layout()
plt.show()

# 성능 향상률 계산
print("\n=== 성능 향상률 ===")


## 9. 임계값 (Threshold) 튜닝

불균형 데이터에서는 기본 임계값 0.5가 최적이 아닐 수 있습니다.

In [ ]:
# 다양한 임계값에서 성능 측정
thresholds = np.arange(0.1, 0.9, 0.05)
f1_scores_by_threshold = []
precision_by_threshold = []
recall_by_threshold = []

for thresh in thresholds:


# 최적 임계값 찾기
best_threshold_idx = np.argmax(f1_scores_by_threshold)
best_threshold = thresholds[best_threshold_idx]
best_f1_threshold = f1_scores_by_threshold[best_threshold_idx]

# F1 score, precision, recall의 최적 임계값 시각화

plt.show()

print(f"\n최적 임계값: { }")
print(f"최적 임계값에서의 F1 Score: { }")

In [ ]:
# 최적 임계값 적용 결과


print("=" * 50)
print(f"최종 결과 (임계값 최적화 적용: {best_threshold:.2f})")
print("=" * 50)
print(classification_report(y_test, final_preds, target_names=['Pass', 'Fail']))

# 최종 Confusion Matrix
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 기존 값을 기반으로 비교 confusion_matrix(y_test, test_preds)

# 변경된 임계값으로 cm 비교 confusion_matrix(y_test, final_preds)



plt.tight_layout()
plt.show()

## 10. 결론 및 요약

In [ ]:
print("=" * 60)
print("   SECOM 불량 예측 - Bayesian Optimization 실습 요약")
print("=" * 60)

print("\n[1] 데이터셋 특성")
print(f"    - 총 샘플: { }%)")
print(f"    - 특성 수: { } (전처리 후)")
print(f"    - 문제점: 극심한 클래스 불균형, 고차원, 결측치")

print("\n[2] Bayesian Optimization 설정")
print(f"    - 방법: Optuna TPE (Tree-structured Parzen Estimator)")
print(f"    - 탐색 횟수: { } trials")
print(f"    - 최적화 목표: Validation F1 Score 최대화")

print("\n[3] 최적 하이퍼파라미터")
print(f"    - 네트워크 구조: { }")
print(f"    - Dropout Rate: { }")
print(f"    - Learning Rate: { }")
print(f"    - Batch Size: { }")

print("\n[4] 최종 성능 (테스트 세트)")
print(f"    - F1 Score: { }")
print(f"    - Precision: { }")
print(f"    - Recall: { }")
print(f"    - ROC-AUC: { }")

print("\n[5] 핵심 인사이트")
print(" ** 직접 작성해보세요 ** ")

print("\n" + "=" * 60)

In [ ]:
print("🎉 고생하셨습니다! 🎉")